In [1]:

# pip3 install cloudscraper beautifulsoup4
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator
from unidecode import unidecode
import json
from datetime import datetime, timedelta
import random
import requests
from langdetect import detect
import numpy as np

In [2]:
def convert_to_camel_case(text):
    # Chuyển tất cả chữ về thường
    text = unidecode(text.lower())
    # Tách các từ dựa trên khoảng trắng
    words = text.split()
    # Viết hoa chữ cái đầu của từng từ và nối chúng lại
    camel_case = ''.join(word.capitalize() for word in words)
    return camel_case

In [3]:
# Hàm để xử lý kiểu dữ liệu không serializable
def default_converter(o):
    if isinstance(o, (np.integer, np.floating)):  # Xử lý kiểu số NumPy
        return int(o)
    elif isinstance(o, (np.ndarray,)):           # Xử lý mảng NumPy (nếu có)
        return o.tolist()
    return str(o)  # Xử lý các kiểu dữ liệu khác

In [4]:
def get_url(soup):
    canonical_tag = soup.find('link', rel='canonical')
    # Lấy giá trị của thuộc tính href
    if canonical_tag and 'href' in canonical_tag.attrs:
        canonical_url = canonical_tag['href']
        return canonical_url
    return None

In [5]:
def get_url_id(url):
    # Regex để tìm ID trong URL
    match = re.search(r"\d+$", url)
    if match:
        return int(match.group())  # Trả về ID tìm thấy
    return None

In [6]:
def get_heading(soup):
    heading1 = soup.find('h1', class_ = "job-title mb-0")
    if heading1:
        content = heading1.get_text(separator=' ', strip=True)
        return content
    return None

In [7]:
def get_ten_cong_ty(soup):
    id = None
    url = None
    name = None
    data = soup.find('p', class_="org-name mb-2")
    if data:
        url = 'https://www.careerlink.vn' + data.find('a').get('href')
        name = data.text.strip()
        if url:
            id = get_url_id(url)
    return url, id, name


In [8]:
def load_df():
    return pd.read_excel(r'C:\Users\My_Pc\Desktop\Thư Viện Pháp Luật final\Crawl tuyển dụng\Code\data_id\coquanhanhchinh.xlsx', engine='openpyxl')


In [9]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhThanh(name, df):
    # print(name)
    result = df[
        df['UnitName'].str.lower().str.contains(name.lower()) &
        (df['ParentId'] == 0) &
        (df['Level'] == 1)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        # parent_id_value = result.iloc[0]['ParentId']
        # level_value = result.iloc[0]['Level']
        name_value = result.iloc[0]['UnitName']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value
    return None, None


In [10]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhQuanHuyen(name, ParentId, df):
    if not name:
        return None, None
    name = name.replace("Quận 2", "Thủ Đức")
    result = df[
        df['UnitName'].str.lower().str.contains(name.lower()) &
        (df['ParentId'] == ParentId) &
        (df['Level'] == 2)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        name_value = result.iloc[0]['UnitName']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value
    return None, None


In [11]:
def get_dia_chi(soup):
    df = load_df()
    output = []
    data = soup.find_all('div', id="job-location")
    if data:
        for temp in data:
            dict = {}
            if temp:
                tinhThanh = temp.find('a', class_="text-reset font-weight-bold")
                if tinhThanh:
                    IdTinhThanh = find_id_tinhThanh(tinhThanh.text.strip(), df)
                    quanHuyen = re.sub(r",\s*$", "", temp.find('span', class_ = "mr-1").text.strip())
                    IdQuanHuyen = find_id_tinhQuanHuyen(quanHuyen, IdTinhThanh[0], df)
                    dict ={
                        'IdTinhThanh': IdTinhThanh[0] or None,
                        'TinhThanh': IdTinhThanh[-1] or None,
                        'IdQuanHuyen': IdQuanHuyen[0] or None,
                        'TinhQuanHuyen':IdQuanHuyen[-1] or None,
                        'DiaChi': None,
                    }
                    output.append(dict)
                else:
                    tinhThanh = temp.find('a', class_="text-reset")
                    if tinhThanh:
                        IdTinhThanh = find_id_tinhThanh(tinhThanh.text.strip(), df)
                        # quanHuyen = re.sub(r",\s*$", "", temp.find('span', class_ = "mr-1").text.strip())
                        # IdQuanHuyen = find_id_tinhQuanHuyen(quanHuyen, IdTinhThanh[0], df)
                        dict ={
                            'IdTinhThanh': IdTinhThanh[0] or None,
                            'TinhThanh': IdTinhThanh[-1] or None,
                            'IdQuanHuyen': None,
                            'TinhQuanHuyen':None,
                            'DiaChi': None,
                        }
                        output.append(dict)
    return output

In [12]:
def get_muc_luong(soup):
    data = soup.find('span', class_="text-primary")
    if data:
        return data.text.strip()
    return None

In [13]:
def get_kinh_nghiem(soup):
    data = soup.find_all('div', class_="d-flex align-items-center mb-2")
    if data:
        for temp in data:
            if temp and temp.find('i', class_="cli-suitcase-simple d-flex mr-2"):
                return temp.text.strip()
    return None

In [14]:
def get_ngay_dang(soup):
    data = soup.find('span', class_="d-none d-md-block mr-1")
    if data:
        return re.search(r"\d{2}-\d{2}-\d{4}", data.parent.text.strip()).group().replace('-', '/')
    return None

In [15]:
def ngay_het_han(soup):
    data = soup.find('div', class_="day-expired d-flex align-items-center")
    if data:
        match = re.search(r"\d+", data.text.strip(), flags=re.IGNORECASE)
        if match:
            return (datetime.now().date() + timedelta(days= int(match.group()))).strftime("%d/%m/%Y")
        elif re.search(r"hôm nay", data.text.strip(), flags=re.IGNORECASE):
            return datetime.now().date().strftime("%d/%m/%Y")
    return None

In [36]:
def get_mo_ta_cong_viec(soup):
    data = soup.find('div', class_="pt-4", id="section-job-description")
    if data:
        return data.find('div', class_="rich-text-content").text.strip()
    # .decode_contents().strip()
    return None

In [41]:
def get_phuc_loi(soup):
    data = soup.find('div', class_="pt-4", id="section-job-benefits")
    if data:
        data1 = data.find_all('img', alt="Benefit Icon")
        if data1:
            for temp in data1:
                temp.decompose()
        return data.find('div', class_="my-3").text.strip()
    # .decode_contents().strip()
    return None

In [37]:
def get_exp_skill(soup):
    data = soup.find('div', class_="raw-content rich-text-content")
    if data:
        return data.text.strip()
    # .decode_contents().strip().text
    return None

In [19]:
def get_mo_ta(soup):
    dict = {}
    data = soup.find_all('div', class_="job-summary-item d-block")
    if data:
        for temp in data:
            dict[convert_to_camel_case(temp.find('div', class_ = "my-0 summary-label").text.strip().replace('\n', ''))] = (temp.find('div', class_ = "font-weight-bolder").text.strip().replace('\n', ''))
        return dict     
    return None   

In [20]:
def get_phone_number(soup):
    data = soup.find_all('li', class_="d-flex align-items-start")
    if data:
        for temp in data:
            if temp and temp.find('i', class_="cli-phone"):
                return temp.find('span', class_="label mr-2 text-nowrap align-seft-center").text.strip()
    return None

In [21]:
def get_lien_he(soup):
    ten_lien_he = None
    dia_chi = None
    ghi_chu = None
    phone_number = get_phone_number(soup) or None
    data = soup.find('ul', class_="list-unstyled contact-person rounded-lg p-3 m-0")
    if data:
        ten_lien_he = data.find('span', class_="person-name font-weight-bolder m-0 align-seft-center").text.strip()
        data1 = data.find_all('li', class_="d-flex align-items-start")
        if data1:
            for temp in data1:
                if temp and temp.find('i', class_ ="cli-location", style="color: #059669; padding-left: 0.25px"):
                    dia_chi = temp.find('span', class_="align-seft-center").text.strip().replace('\n', '')
        ghi_chu = data.find('div', class_="rich-text-content").decode_contents().strip()
    dict = {
        'TenLienHe': ten_lien_he,
        'SoDienThoai': phone_number,
        'DiaChi': dia_chi,
        'GhiChu': ghi_chu
    }
    return dict


In [22]:
def get_key_word(soup):
    output = []
    data = soup.find_all('a', class_="chip border-0 mr-2 mb-2")
    if data:
        for temp in data:
            output.append(temp.text.strip())
    return output

In [23]:
def convert_value(value):
    if not value or value == 'None':
        return None
    try:
        # Kiểm tra nếu là số nguyên
        return int(value)
    except ValueError:
        try:
            # Kiểm tra nếu là số thập phân
            return float(value)
        except ValueError:
            # Nếu không phải số, trả về dưới dạng chuỗi
            return str(value)


In [24]:
def convert_number(value):
    # try:
        if not value:
            return None
        number = float(value.replace(",", ""))
        # Kiểm tra nếu là số nguyên
        if number.is_integer():
            return int(number)
        else:
            return number  # Trả về float
    # except Exception:
    #     return value  # Trả về giá trị gốc nếu có lỗi

In [25]:
def extract_number_salary(text):
    # Biểu thức chính quy tìm kiếm số, bao gồm số có dấu phẩy phân cách
    match = re.search(r'[\d,]+', text)
    if match:
        # Loại bỏ dấu ',' và trả về số dưới dạng float
        number = match.group(0)
        return convert_number(number)  # Chuyển đổi thành float
    return None

In [26]:
def processing_salary(text):
    if 'triệu' in text:
        don_vi_tien = 'VND'
        muc_luong = text.replace('triệu', '')
    elif 'USD' in text:
        don_vi_tien = 'USD'
        muc_luong = text.replace('USD', '')
    else:
        don_vi_tien = None
        muc_luong = text
    muc_luong = muc_luong.split('-')
    if len(muc_luong) > 1:
        muc_luong_den = extract_number_salary(muc_luong[1].strip())
        muc_luong_tu = extract_number_salary(muc_luong[0].strip())
    else:
        if muc_luong[0].strip().lower().startswith('trên') or muc_luong[0].strip().lower().startswith('từ'):
            muc_luong_den = 0
            muc_luong_tu = extract_number_salary(muc_luong[0])
        elif muc_luong[0].strip().lower().startswith('dưới') or muc_luong[0].strip().lower().startswith('tới'):
            muc_luong_tu = 0
            muc_luong_den = extract_number_salary(muc_luong[0])
        else:
            muc_luong_tu = 0
            muc_luong_den = 0
    return muc_luong_tu, muc_luong_den, don_vi_tien

In [27]:
def IsThoaThuan(text):
    if processing_salary(text) == (0, 0, None):
        return True
    return False

In [ ]:
output = []
input_data = []
for temp in range(0,1000):
    while True:
        try:
            ids = 3022174 - temp
            scraper = cloudscraper.create_scraper(
                            browser={
                                "browser": "chrome",
                                "platform": "windows",
                            },
                        )
            # specify the target URL
            url = f"https://www.careerlink.vn/tim-viec-lam/truong-phong-kinh-doanh/{ids}"
            time.sleep(3)
            response = scraper.get(url)
            print(response.status_code)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                if get_heading(soup):
                    data = {
                        'JobId': convert_value(get_url_id(get_url(soup))),
                        'UrlJob': convert_value(get_url(soup)),
                        'JobTitle': convert_value(get_heading(soup)),
                        'IdCongTy': convert_value(get_ten_cong_ty(soup)[1]), 
                        'UrlCongTy': convert_value(get_ten_cong_ty(soup)[0]),
                        'TenCongTy': convert_value(get_ten_cong_ty(soup)[2]),
                        'DiaDiemLamViec': get_dia_chi(soup),
                        'IsThoaThuan': IsThoaThuan(get_muc_luong(soup)),
                        'mucluong': get_muc_luong(soup),
                        'mucluongfrom': processing_salary(get_muc_luong(soup))[0],
                        'mucluongto': processing_salary(get_muc_luong(soup))[1],
                        'currency': processing_salary(get_muc_luong(soup))[2],
                        'KinhNghiem': convert_value(get_kinh_nghiem(soup)),
                        'NgayDangTin': get_ngay_dang(soup),
                        'HanNopHoSo': ngay_het_han(soup),
                        'MoTaCongViec': get_mo_ta_cong_viec(soup),
                        'QuyenLoi': get_phuc_loi(soup),
                        'YeuCauKyNang': get_exp_skill(soup),
                        'MoTaTomTat': get_mo_ta(soup),
                        'ThongTinLienHe': get_lien_he(soup),
                        'TuKhoa': get_key_word(soup),
                    }
                    data_input = (get_heading(soup)) + '\n' + (get_ten_cong_ty(soup)[2]) +  ' tuyển ' + (get_heading(soup)) + ', mức lương ' + get_muc_luong(soup) + ', kinh nghiệm ' + (get_kinh_nghiem(soup)) + '\n' + 'Mô tả công việc: ' + '\n' + get_mo_ta_cong_viec(soup) + '\nQuyền lợi\n' + get_phuc_loi(soup) + '\nYêu cầu\n' + get_exp_skill(soup)
                    # print(data_input)
                    input_data.append(data_input)
                    output.append(data)
                    # print(json.dumps(data, ensure_ascii=False, indent=5, default=default_converter))
                    print(response.status_code, "Cập nhật thông tin thành công.", ids, len(output))
            if response.status_code == 429:
                    # print(f"ID {ids}: Too many requests, retrying...")
                time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
            else:
                break  # Thoát khỏi vòng lặp nếu không phải 429
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
            time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại

200
200 Cập nhật thông tin thành công. 3022174 1
404
404
404
404


In [ ]:
data_set

{'CapBac',
 'GioiTinh',
 'HocVan',
 'KinhNghiem',
 'LoaiCongViec',
 'MaViecLam',
 'NganhNghe',
 'Tuoi'}

In [ ]:
# # Lưu vào tệp JSON
with open(r"C:\Users\My_Pc\Desktop\Thư Viện Pháp Luật final\Crawl tuyển dụng\output_sample\sample_careerlink_cv_v3.json", "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, ensure_ascii=False, indent=5, default=default_converter)